<a href="https://colab.research.google.com/github/ZzangJay/team_project/blob/kimjisan/221107_extra_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
#import tensorflow as tf 
import os
import sys
from datetime import datetime, timedelta
from tqdm.autonotebook import tqdm
import csv
import re
import gzip
import shutil

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import sys


In [2]:
from google.colab import drive  # for colab
drive.mount('/content/gdrive/')  # for colab
cloud_directory = '/content/gdrive/MyDrive/mimiciv_unzip/'
os.listdir(cloud_directory) # for colab, check cloud directory mount

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


['CHANGELOG.txt',
 'LICENSE.txt',
 'SHA256SUMS.txt',
 'index.html',
 'icu',
 'hosp',
 'drop_2_chartevents.csv']

In [3]:
sepsis = pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/sepsis3.csv', low_memory=False, dtype='str', keep_default_na=False)
urine = pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/urine_output_rate.csv', low_memory=False, dtype='str', keep_default_na=False)
age = pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/age.csv', low_memory=False, dtype='str', keep_default_na=False)
weight = pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/weight_durations.csv', low_memory=False, dtype='str', keep_default_na=False)
vaso=pd.read_csv('/content/gdrive/MyDrive/mimiciv_der/vasoactive_agent.csv', low_memory=False, dtype='str', keep_default_na=False)
admission = pd.read_csv('/content/gdrive/MyDrive/mimiciv_unzip/hosp/admissions.csv', low_memory=False, dtype='str', keep_default_na=False)

In [4]:
# vaso

In [5]:
admission = pd.merge(left = admission, right = sepsis[['subject_id','stay_id','sepsis3']], how='inner', left_on='subject_id', right_on='subject_id')
urine = pd.merge(left = urine, right = admission[['subject_id', 'hadm_id','stay_id']], how='inner', left_on='stay_id', right_on='stay_id')
age = pd.merge(left = age, right = admission[['subject_id', 'hadm_id','stay_id']], how='inner', left_on = ['hadm_id','subject_id'], right_on = ['hadm_id', 'subject_id'])


In [6]:
# urine=urine[['subject_id', 'hadm_id', 'weight', 'urineoutput_6hr', 'urineoutput_12hr', 'urineoutput_24hr']]
urine=urine[['subject_id', 'hadm_id', 'weight']]

In [7]:
extra_features = pd.merge(left = urine, right = age[['subject_id', 'hadm_id','age']], how='inner', left_on=['subject_id', 'hadm_id'], right_on=['subject_id', 'hadm_id'])

In [14]:
extra_features = extra_features.drop_duplicates(subset=['subject_id','hadm_id'], keep='first')

In [17]:
extra_features['age']=pd.to_numeric(extra_features['age'])
extra_features['weight']=pd.to_numeric(extra_features['weight'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
extra_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105622 entries, 0 to 37015877
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   subject_id  105622 non-null  object 
 1   hadm_id     105622 non-null  object 
 2   weight      104408 non-null  float64
 3   age         105622 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 4.0+ MB


In [19]:
extra_features=extra_features.dropna(subset=['weight'], axis=0)

In [20]:
extra_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104408 entries, 0 to 37015876
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   subject_id  104408 non-null  object 
 1   hadm_id     104408 non-null  object 
 2   weight      104408 non-null  float64
 3   age         104408 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 4.0+ MB


In [21]:
extra_features.to_csv('/content/gdrive/MyDrive/mimiciv preprocessing_js/extra_features.csv',index=False)